In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/mnt/nvme_share/cuizy/models/Qwen-Audio"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda:4", trust_remote_code=True, bf16=True, fp32=False)
# model.eval()
# transformer = model.transformer

/home/cuizy/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


audio_start_id: 155163, audio_end_id: 155164, audio_pad_id: 151851.


In [5]:
audio_url = "/mnt/nvme_share/cuizy/SA-detection/audio/02/9-4-lxy-02.wav"
prompt = "Is there a suicide risk?"
query = f"<audio>{audio_url}</audio>{prompt}"
audio_info = tokenizer.process_audio(query)
inputs = tokenizer(query, return_tensors="pt", audio_info=audio_info)
print(audio_info)
print(inputs)

{'input_audios': tensor([[[-0.5019, -0.3529, -0.2185,  ..., -1.4261, -1.4261, -1.4261],
         [-0.4347, -0.2462, -0.3575,  ..., -1.4261, -1.4261, -1.4261],
         [-0.1445, -0.2591, -0.3669,  ..., -1.4261, -1.4261, -1.4261],
         ...,
         [-0.9120, -0.8100, -0.8597,  ..., -1.4261, -1.4261, -1.4261],
         [-0.9649, -0.9365, -0.9005,  ..., -1.4261, -1.4261, -1.4261],
         [-0.9714, -1.0208, -1.0564,  ..., -1.4261, -1.4261, -1.4261]]]), 'input_audio_lengths': tensor([[1495,  747]], dtype=torch.int32), 'audio_span_tokens': [749], 'audio_urls': ['/mnt/nvme_share/cuizy/SA-detection/audio/02/9-4-lxy-02.wav']}
{'input_ids': tensor([[155163, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 15185

In [2]:
audio_url1 = "/mnt/nvme_share/cuizy/SA-detection/audio/02/0001-1-zyh-02.wav"
audio_url2 = "/mnt/nvme_share/cuizy/SA-detection/audio/02/1-3-cw-02.wav"
prompt = "Is there a suicide risk?"
query1 = f"<audio>{audio_url1}</audio>{prompt}"
query2 = f"<audio>{audio_url2}</audio>{prompt}"
# tokenizer.pad_token = tokenizer.eos_token

# transformer = model.transformer
with torch.no_grad():
    audio_info = tokenizer.process_audio(query1)
    inputs = tokenizer(query1, return_tensors="pt", audio_info=audio_info)
    # inputs = inputs.to(model.device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    # output = transformer(input_ids, audio_info=audio_info)
    # pred = model.generate(**inputs, audio_info=audio_info)
    # response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False,audio_info=audio_info)

In [3]:
audio_info1 = tokenizer.process_audio(query1)
audio_info2 = tokenizer.process_audio(query2)
print(audio_info1)
print(audio_info1['input_audios'].shape)
print(audio_info2)
print(audio_info2['input_audios'].shape)

{'input_audios': tensor([[[ 0.6029,  0.5021,  0.5876,  ..., -1.0556, -1.0556, -1.0556],
         [ 0.3328,  0.2430,  0.4369,  ..., -1.0556, -1.0556, -1.0556],
         [ 0.0938,  0.2818,  0.3794,  ..., -1.0556, -1.0556, -1.0556],
         ...,
         [-0.2480, -0.2655, -0.3047,  ..., -1.0556, -1.0556, -1.0556],
         [-0.3987, -0.3387, -0.3805,  ..., -1.0556, -1.0556, -1.0556],
         [-0.4669, -0.5562, -0.5340,  ..., -1.0556, -1.0556, -1.0556]]]), 'input_audio_lengths': tensor([[296, 148]], dtype=torch.int32), 'audio_span_tokens': [150], 'audio_urls': ['/mnt/nvme_share/cuizy/SA-detection/audio/02/0001-1-zyh-02.wav']}
torch.Size([1, 80, 3000])
{'input_audios': tensor([[[-0.1401,  0.0179,  0.2169,  ..., -1.1235, -1.1235, -1.1235],
         [ 0.1653, -0.0783,  0.1523,  ..., -1.1235, -1.1235, -1.1235],
         [ 0.3062, -0.0163,  0.1312,  ..., -1.1235, -1.1235, -1.1235],
         ...,
         [-0.7244, -0.6081, -0.5282,  ..., -1.1235, -1.1235, -1.1235],
         [-0.6638, -0.5930

In [5]:
audio_info['input_audios'] = torch.concat((audio_info1['input_audios'], audio_info2['input_audios']), dim=0)
audio_info['input_audio_lengths'] = torch.concat((audio_info1['input_audio_lengths'], audio_info2['input_audio_lengths']), dim=0)
audio_info['audio_span_tokens'] = audio_info1['audio_span_tokens']
audio_info['audio_span_tokens'].extend(audio_info2['audio_span_tokens'])
audio_info['audio_urls'] = audio_info1['audio_urls']
audio_info['audio_urls'].extend(audio_info2['audio_span_tokens'])

inputs1 = tokenizer(query1, return_tensors="pt", audio_info=audio_info1)
inputs2 = tokenizer(query2, return_tensors="pt", audio_info=audio_info2)
# input_ids = torch.nn.utils.rnn.pad_sequence([inputs1["input_ids"].squeeze(), inputs2["input_ids"].squeeze()]).to(model.device).transpose(0, 1)
input_ids = torch.nn.utils.rnn.pad_sequence([inputs1["input_ids"].squeeze(), inputs2["input_ids"].squeeze()]).transpose(0, 1)

In [9]:
print(input_ids.shape)
print(audio_info['input_audios'].shape)
print(audio_info1['input_audios'].shape)
print(audio_info['audio_span_tokens'])
print(audio_info2['audio_span_tokens'])

torch.Size([2, 197])
torch.Size([2, 80, 3000])
torch.Size([1, 80, 3000])
[150, 191, 191]
[191]


In [11]:
output = transformer(input_ids, audio_info=audio_info)

In [12]:
print(output.last_hidden_state.shape)

torch.Size([2, 197, 4096])


In [25]:
print(inputs)
input_lens = (input_ids != 0).sum(dim=-1)
print(input_ids.shape, input_ids.dtype, input_lens)
print(output.last_hidden_state.shape)

{'input_ids': tensor([[155163, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851, 151851,
         15185